In [23]:
from AnthropicWrapper import ClaudeChat, ClaudeChatCV
from HumeWrapper import HumeSentimentAnalyzer
#import conversationVerifier
from dotenv import load_dotenv
from joblib import load
import os

In [24]:
# -- Global Variables --
timestamp = 1723589672
chatlog = load(f'interviews/{timestamp}/joblib/conversation.joblib') # Loading in the list of dicts

In [25]:
def reformat_chatlog(chatlog):
    dropped_context = chatlog[3:]
    outputchatlog = []

    for i in range(0, len(dropped_context), 2):
        if i + 1 < len(dropped_context):
            tempdict = {
                'interviewer': dropped_context[i]['content'],
                'candidate': dropped_context[i+1]['content']
            }

            outputchatlog.append(tempdict)
        else:
            tempdict = {
                'interviewer': dropped_context[i]['content'],
                'candidate': ''  
            }
            outputchatlog.append(tempdict)
            break 

    return outputchatlog

chatlog_chat = reformat_chatlog(chatlog)
# for item in chatlog_chat:
#     print(item)

In [26]:
dotenv_path = os.path.join(os.path.dirname(os.getcwd()), ".env")
load_dotenv(dotenv_path)

True

In [37]:
# Initialising the Sentiment Analyser
sentiment_analyser = HumeSentimentAnalyzer(api_key=os.getenv("HUME_API_KEY"))

In [38]:
# Initialising Sentiment Summariser
chat_model_name = "claude-3-5-sonnet-20240620"

system_prompt = """You are a skilled emotions analyst provided with a detailed breakdown of sentiment analysis scores from Hume.ai, for a single response in an interview to a question from the interviewer. The scores are split into 3 sections. All numbers are from 0 to 1, linearly scaling, with 1 being a very strong representation of the indicator in question.

First, Emotions. This contains several human emotions with a numerical value indicating the strength of the corresponding emotion.
Second, Sentiments. This contains a scale from 1 to 9, each containing a numerical value indicating the magnitude of the sentiment of the topic of the conversation. A negative topic such as murder will have a high value lower in the scale, such as 1 or 2, and a positive topic will have a high value from 0 to 1 higher in the scale such as 8 or 9.
Third, Toxicity. This contains several toxic representations such as hate, insult, etc, with a value from 0 to 1 for each representation identified in the audio.

Your job is to provide a concise detailed one sentence breakdown of how the individual was feeling for the particular scores provided. You must be highly objective as your job is to discern whether or not a candidate was exhibiting traits which would or would not be fitting for a successful interview. 
Model your answer beginning with something along the lines of "For this particular response, the candidate...".
"""

sentiment_summariser = ClaudeChat(chat_model_name, system_prompt)

In [ ]:
# Initialising verifier
#inaccuracies = conversationVerifier.process_qa_pair(chatlog[3], chatlog[4])

In [39]:
# Initialising Evaluation LLM

chat_model_name = "claude-3-5-sonnet-20240620"

system_prompt = "Claude, write me a sonnet about the beauty of the universe."

candidate_evaluator = ClaudeChat(chat_model_name, system_prompt)

In [43]:
import os

# Specify the filepath
filepath = f'interviews/{timestamp}/audio'


# Get all files in the filepath
files = [f for f in os.listdir(filepath) if os.path.isfile(os.path.join(filepath, f))]

# Loop through each file
sentiments = []
for count, file in enumerate(files, 1):
    # Print the file name

    result = sentiment_analyser.analyze_audio(os.path.join(filepath, str(file)))
    sentiments.append(result)
    print(file)


Analyzing audio...
audio_1_1723589672.wav
Analyzing audio...
audio_2_1723589672.wav


In [44]:
for sentiment in sentiments:
    print(sentiment)

{'emotions': {'Admiration': 0.0718897134065628, 'Adoration': 0.016121450811624527, 'Aesthetic Appreciation': 0.03637176752090454, 'Amusement': 0.03838168457150459, 'Anger': 0.00028867661603726447, 'Annoyance': 0.003975479863584042, 'Anxiety': 0.0010712327202782035, 'Awe': 0.012564522214233875, 'Awkwardness': 0.019182339310646057, 'Boredom': 0.026648489758372307, 'Calmness': 0.4037717878818512, 'Concentration': 0.02281680516898632, 'Confusion': 0.004914105869829655, 'Contemplation': 0.049012910574674606, 'Contempt': 0.017676912248134613, 'Contentment': 0.19272065162658691, 'Craving': 0.0006586601375602186, 'Desire': 0.0002682713675312698, 'Determination': 0.0063696494325995445, 'Disappointment': 0.0013081046054139733, 'Disapproval': 0.002958880504593253, 'Disgust': 0.0008353133453056216, 'Distress': 0.000400354532757774, 'Doubt': 0.003821812802925706, 'Ecstasy': 0.0012174175353720784, 'Embarrassment': 0.0014594719978049397, 'Empathic Pain': 0.0027105743065476418, 'Enthusiasm': 0.1882288